In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow.keras.utils as ku
import numpy as np

In [7]:
data = open('poetry.txt').read()
corpus = data.lower().split("\n")

print(corpus[0])

from fairest creatures we desire increase,


In [46]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)

total_words = len(tokenizer.word_index)+1

print(total_words)
input_sequences =[]

for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    #print(token_list)
    for i in range(1,len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)
#print(input_sequences)
    
    
max_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences,maxlen=max_len,padding='pre'))


predictions,label = input_sequences[:,:-1],input_sequences[:,-1]

label = ku.to_categorical(label,num_classes=total_words)

3211


In [47]:
print(predictions.shape)
#np.expand_dims(predictions,axis=2)
#np.expand_dims(label,axis=2)
print(predictions.shape)

(15462, 10)
(15462, 10)


In [82]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(total_words,32,input_length=max_len-1),
    tf.keras.layers.Bidirectional(LSTM(32,input_shape=(32,10),return_sequences=True)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Bidirectional(LSTM(32)),
    tf.keras.layers.Dense(64,activation='relu'),
    tf.keras.layers.Dense(total_words,activation='softmax')
])


In [83]:
model.summary()

Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_27 (Embedding)     (None, 10, 32)            102752    
_________________________________________________________________
bidirectional_42 (Bidirectio (None, 10, 64)            16640     
_________________________________________________________________
dropout_25 (Dropout)         (None, 10, 64)            0         
_________________________________________________________________
bidirectional_43 (Bidirectio (None, 64)                24832     
_________________________________________________________________
dense_44 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_45 (Dense)             (None, 3211)              208715    
Total params: 357,099
Trainable params: 357,099
Non-trainable params: 0
_______________________________________________

In [84]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [87]:
model.fit(predictions,label,epochs=100)

Train on 15462 samples
Epoch 1/100
15462/15462 [==============================] - 11s 715us/sample - loss: 5.4262 - accuracy: 0.0558
Epoch 2/100
15462/15462 [==============================] - 11s 711us/sample - loss: 5.3176 - accuracy: 0.0627
Epoch 3/100
15462/15462 [==============================] - 11s 734us/sample - loss: 5.2208 - accuracy: 0.0627
Epoch 4/100
15462/15462 [==============================] - 11s 731us/sample - loss: 5.1205 - accuracy: 0.0676
Epoch 5/100
15462/15462 [==============================] - 11s 733us/sample - loss: 5.0131 - accuracy: 0.0726
Epoch 6/100
15462/15462 [==============================] - 11s 728us/sample - loss: 4.9134 - accuracy: 0.0761
Epoch 7/100
15462/15462 [==============================] - 11s 727us/sample - loss: 4.8112 - accuracy: 0.0774
Epoch 8/100
15462/15462 [==============================] - 12s 764us/sample - loss: 4.7066 - accuracy: 0.0854
Epoch 9/100
15462/15462 [==============================] - 12s 786us/sample - loss: 4.6077 - accu

15462/15462 [==============================] - 12s 793us/sample - loss: 1.9664 - accuracy: 0.5179
Epoch 75/100
15462/15462 [==============================] - 12s 790us/sample - loss: 1.9478 - accuracy: 0.5232
Epoch 76/100
15462/15462 [==============================] - 12s 790us/sample - loss: 1.9471 - accuracy: 0.5235
Epoch 77/100
15462/15462 [==============================] - 12s 802us/sample - loss: 1.9293 - accuracy: 0.5252
Epoch 78/100
15462/15462 [==============================] - 12s 798us/sample - loss: 1.9122 - accuracy: 0.5285
Epoch 79/100
15462/15462 [==============================] - 13s 814us/sample - loss: 1.9135 - accuracy: 0.5314
Epoch 80/100
15462/15462 [==============================] - 17s 1ms/sample - loss: 1.8956 - accuracy: 0.5363
Epoch 81/100
15462/15462 [==============================] - 13s 814us/sample - loss: 1.8516 - accuracy: 0.5444
Epoch 82/100
15462/15462 [==============================] - 13s 826us/sample - loss: 1.8574 - accuracy: 0.5413
Epoch 83/100
154

In [88]:
seed_text = "That thereby beauty's rose might never die"
next_words = 100
  
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_len-1, padding='pre')
    predicted = model.predict_classes(token_list, verbose=0)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word
print(seed_text)

That thereby beauty's rose might never die be shown find room hang clears night back still ' ' your might bring all live thee so not so thou taste not word doth true stand on your glory alone green me thee store ' room room how there of word lies right you thy clock for thine eye by me ever thee alone thee you know so eyes me due of thee thy sorrows parts decay show thee thee room room room room room room room room none of heart thy wit had dumb sweetness set it dead ' you are hate lies none none bright praise doth
